In [1]:
# Collaborative Filtering- makes recommendations based on the idea that users who agreed in the past will agree in the future.

# It can be user-based (finding similar users) or item-based (finding similar items). This notebook uses item-based collaborative filtering to find movies similar to a target movie based on user ratings.

#The goal of this notebook is to create a collaborative filtering-based recommender system using movie rating data. 
import pandas
user_item_details = pandas.read_csv("https://modcom.co.ke/datasets/file.csv")
user_item_details.head(2)

,user_id,item_id,rating,timestamp
0,0,50,5,881250949
1,0,172,5,881250949


In [2]:
movie_details = pandas.read_csv("https://modcom.co.ke/datasets/Movie_Id_Titles.csv")
movie_details.head(2)

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)


In [3]:
# merge the two datasets
data = pandas.merge(user_item_details, movie_details, on = 'item_id')
data.head()

,user_id,item_id,rating,timestamp,title
0,0,50,5,881250949,Star Wars (1977)
1,0,172,5,881250949,"Empire Strikes Back, The (1980)"
2,0,133,1,881250949,Gone with the Wind (1939)
3,196,242,3,881250949,Kolya (1996)
4,186,302,3,891717742,L.A. Confidential (1997)


In [4]:
# Find average rating per movie.Displays the top movies with the highest average ratings.
# Calculate the average rating for each movie
#This step is crucial for identifying top-rated movies and understanding user preferences.
data.groupby('title')['rating'].mean().sort_values(ascending=False).head(5)

title
They Made Me a Criminal (1939)                5.0
Marlene Dietrich: Shadow and Light (1996)     5.0
Saint of Fort Washington, The (1993)          5.0
Someone Else's America (1995)                 5.0
Star Kid (1997)                               5.0
Name: rating, dtype: float64

In [5]:
# How many times was it rated
# Find average rating per movies
data.groupby('title')['rating'].count().sort_values(ascending=False).tail(5)

title
Great Day in Harlem, A (1994)         1
Other Voices, Other Rooms (1997)      1
Good Morning (1971)                   1
Girls Town (1996)                     1
Á köldum klaka (Cold Fever) (1994)    1
Name: rating, dtype: int64

In [6]:
#We want to show the  number of ratings for each movie.We show the first 5
#The goal of the code is to create a DataFrame that shows the average rating for each movie along with the number of ratings each movie has received. 
#We are creating a new table that consists of title, average rating and the number of ratings(how many times was it rated).  
mean_ratings = pandas.DataFrame(data.groupby('title')['rating'].mean())
mean_ratings['num_of_ratings'] = pandas.DataFrame(data.groupby('title')['rating'].count())
mean_ratings.head(5)


,rating,num_of_ratings
title,,
'Til There Was You (1997),2.333333,9
1-900 (1994),2.600000,5
101 Dalmatians (1996),2.908257,109
12 Angry Men (1957),4.344000,125
187 (1997),3.024390,41


In [7]:
# Find which movie did each user rate, and what was the rating  -PIVOT TABLE
#A pivot table is a powerful data summarization tool that is used in data processing to organize, group, and aggregate data.
pivot   = data.pivot_table(index = 'user_id', columns = 'title', values = 'rating')
pivot.head(5)

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Lets correlate movies using the pivot
#Finding movie correlations in the context of a recommender system helps us identify how similar the rating patterns of different movies are. 
selected_movie = pivot['Godfather, The (1972)']

# find similar movies to this one based ratings

similar = pivot.corrwith(selected_movie)
similar_df  = pandas.DataFrame(similar, columns = ['Correlations'])
similar_df.sort_values('Correlations', ascending = False).head(50)
#A correlation value of 1 means there is a perfect positive correlation between the ratings of two movies.
#A correlation value of 0 means there is no correlation between the ratings of two movies. The ratings of one movie do not predict the ratings of the other movie. This suggests no particular relationship in terms of user preferences.
#A correlation value of -1 means there is a perfect negative correlation between the ratings of two movies. When users rate one movie highly, they rate the other movie poorly, and vice versa.


c:\Users\Shanice\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\Shanice\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Shanice\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\Shanice\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
c:\Users\Shanice\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


,Correlations
title,
Dark City (1998),1.000000
8 Seconds (1994),1.000000
Talking About Sex (1994),1.000000
"Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)",1.000000
"Bye Bye, Love (1995)",1.000000
"Locusts, The (1997)",1.000000
Kicked in the Head (1997),1.000000
"Outlaw, The (1943)",1.000000
"Beans of Egypt, Maine, The (1994)",1.000000


In [9]:
# Show by movies that only had over 200 ratings
#Adding another column
similar_df = similar_df.join(mean_ratings['num_of_ratings'])
similar_df.head(5)

,Correlations,num_of_ratings
title,,
'Til There Was You (1997),0.612372,9
1-900 (1994),-0.471405,5
101 Dalmatians (1996),0.084183,109
12 Angry Men (1957),0.034258,125
187 (1997),0.467335,41


In [10]:
# Take only the one with over 200 ratings
similar_df[similar_df['num_of_ratings'] > 200].sort_values('Correlations', ascending = False).head(20)

,Correlations,num_of_ratings
title,,
"Godfather, The (1972)",1.000000,413
"Godfather: Part II, The (1974)",0.683862,209
GoodFellas (1990),0.421477,226
"People vs. Larry Flynt, The (1996)",0.393439,215
Apocalypse Now (1979),0.374378,221
Dead Man Walking (1995),0.360525,299
Psycho (1960),0.336903,239
Field of Dreams (1989),0.309903,212
2001: A Space Odyssey (1968),0.305717,259
